In [1]:
import pandas as pd
import numpy as np

# Read.
df = pd.read_excel('reuben_calendar_report.xls')

# Extract only the useful columns.
df = df[['Text','Weekday','Date','Duration']]

# Drop everywhere without any actual tutoring data.
df = df[pd.notnull(df['Duration'])]

# Drop rows with NaN values in them.
df = df.dropna()

# Make timestamp objects.
df['Date'] = [pd.Timestamp(x) for x in df['Date']]

# Reset index.
df.reset_index(drop=True,inplace=True)

## Tutoring Events Only

In [2]:
# Extract only sessions with 'TUTOR' in the name.
tut = df[df['Text'].str.contains('TUTOR')]

# Split each TEXT column element and extract the student names.
tut['Student'] = [element.split()[1] for element in tut['Text']]

/home/reuben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [3]:
# Remap the names to account for slight mis-spellings or things like
# "so and so's daughter/son".

name_remap_dict = {'KELLY\'S':'KENEDY','SHAUNA\'S':'KRISTIANA','HAEMAN':'HARMAN',
                      'KARA\'S':'KEEGEN','MOHAMMED':'MOHAMMAD'}

updated_names = [name_remap_dict[name] if name in name_remap_dict.keys() 
                 else name for name in tut['Student']]

tut.loc[:,'Student'] = updated_names

/home/reuben/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
# Remove the 'text' column from the DataFrame.
tut.drop('Text',axis=1,inplace=True)

# Change duration to a float value.
def convert_duration(timestamp):
    """Convert timestamp of duration into a float number of hours."""
    dur_split = str(timestamp).split(':')
    dur = float(dur_split[0])+float(dur_split[1])/60.0
    return(dur)

tut['Duration'] = [convert_duration(x) for x in tut['Duration']]

/home/reuben/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/reuben/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [21]:
# Output the cleaned datafile.
tut.to_csv('cleaned_tutoring_data.csv',index=False)